In [1]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import autogluon.core as ag
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error
import torch


c:\Users\Alex\miniconda3\envs\env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0

True
1


In [3]:
df_avg = pd.read_pickle('../Data/ELC_12-14_1D_dropped_avg.pkl')

df_avg_convert = df_avg.copy()
df_avg_convert['id'] = 1
df_avg_convert = df_avg_convert.rename(columns = {"y": "target"})
df_avg_convert = TimeSeriesDataFrame.from_data_frame(
    df_avg_convert,
    id_column="id",
    timestamp_column="ds"
)

df_avg_convert.head()

target
item_id timestamp               
1       2012-01-01  38313.751557
        2012-01-02  57225.098775
        2012-01-03  57294.353754
        2012-01-04  57503.687632
        2012-01-05  57662.515150

In [4]:
training_x = df_avg_convert.iloc[:int(len(df_avg_convert)*0.8)]
val_x = df_avg_convert.iloc[int(len(df_avg_convert)*0.8):int(len(df_avg_convert)*0.9)]
testing_x = df_avg_convert.iloc[int(len(df_avg_convert)*0.9):]
print("training: ", training_x.shape)
print("validation: ", val_x.shape)
print("testing: ", testing_x.shape)

training:  (876, 1)
validation:  (110, 1)
testing:  (110, 1)


### TFT

In [5]:
predictor_TFT = TimeSeriesPredictor(prediction_length = 110, eval_metric="MAPE", fast_training = True)

predictor_TFT.fit(
    training_x,
    hyperparameters={
        "TemporalFusionTransformer":{}
    },
    tuning_data = val_x,
    ag_args_fit={'num_gpus': 1}
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'ag_args_fit': {'num_gpus': 1},
 'enable_ensemble': True,
 'evaluation_metric': 'MAPE',
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': {'TemporalFusionTransformer': {}},
 'prediction_length': 110,
 'random_seed': None,
 'target': 'target',
 'time_limit': None}
Provided training data set with 876 rows, 1 items (item = single time series). Average time series length is 876.0.
Provided tuning data set with 110 rows, 1 items. Average time series length is 110.0.
Training artifacts will be saved to: C:\Users\Alex\OneDrive - University of Rochester\Desktop\2023Spring\Forcasting\IEOR4574-Forcasting\Electricity_Forecasting\Model\AutogluonModels\ag-20230502_022829
AutoGluon will save models to AutogluonModels\ag-20230502_022829\
AutoGluon will gauge predictive performance using evaluation metric: 'MAPE'
	This metric's sign has been flipped to adhere to being 'higher is bette

In [10]:
predictions_TFT = predictor_TFT.predict(pd.concat(training_x, val_x))
predictions_TFT.head(5)

C:\Users\Alex\AppData\Local\Temp\ipykernel_25336\1942128367.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  predictions_TFT = predictor_TFT.predict(pd.concat(training_x, val_x))


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "TimeSeriesDataFrame"

In [6]:
predictor_DAR = TimeSeriesPredictor(prediction_length = 110, eval_metric="MAPE", fast_training = True)

predictor_DAR.fit(
    testing_x,
    hyperparameters={
        "DeepAR":{}
    },
    tuning_data = val_x,
    ag_args_fit={'num_gpus': 1}
)

No path specified. Models will be saved in: "AutogluonModels\ag-20230502_024424\"
c:\Users\Alex\miniconda3\envs\env2\lib\site-packages\autogluon\timeseries\predictor.py:436: UserWarning: Detected short time series in train_data. For best performance, all training time series should have length >= 2 * prediction_length + 1(at least 221).
  warnings.warn(
================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'ag_args_fit': {'num_gpus': 1},
 'enable_ensemble': True,
 'evaluation_metric': 'MAPE',
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': {'DeepAR': {}},
 'prediction_length': 110,
 'random_seed': None,
 'target': 'target',
 'time_limit': None}
Provided training data set with 110 rows, 1 items (item = single time series). Average time series length is 110.0.
Provided tuning data set with 110 rows, 1 items. Average time series length is 110.0.
Training artifacts will be saved to: C:\Users\Alex\OneDrive - University of Ro